In [18]:
from agent import Agent
from referee.game import PlayerColor
from agent.board import MatrixBoard
import numpy as np
from referee.game import \
    PlayerColor, Action, SpawnAction, SpreadAction, HexPos, HexDir
import math
from enum import Enum


In [19]:
# Function to update the r,q if they reach -1 or 7
def update_r_q(r, q, dr, dq) -> tuple:
    """
    Update the coordinates if the token reaches the edge.
    """
    r += dr
    q += dq
    if r > 6:
        r = 0
    if r < 0:
        r = 6
    if q > 6:
        q = 0
    if q < 0:
        q = 6
    return (r, q)

In [2]:
np_empty = np.zeros(shape=[2,7,7], dtype=int)
board_empty = MatrixBoard(np_empty, 0, 0, 0)

In [6]:
np_more_red = np.array([[[0,0,1,3,5,0,0], 
                        [0,1,0,0,0,3,0],
                        [0,0,5,2,0,0,0],
                        [1,2,0,0,3,0,4],
                        [3,0,1,0,0,0,0],
                        [0,2,1,0,0,0,0],
                        [1,0,0,2,0,4,6]],
                        [[1,2,0,0,0,0,0],
                         [3,0,0,0,0,0,0],
                         [4,0,0,0,0,0,0],
                         [0,0,0,0,0,0,0],
                         [0,0,0,0,0,3,0],
                         [0,0,0,0,0,0,0],
                         [0,0,0,0,5,0,0]]])
board_mroe_red = MatrixBoard(np_more_red, 100, np_more_red[0].sum(), np_more_red[1].sum())

In [9]:
print(board_mroe_red.render(use_color=True, use_unicode=True))

                         r1     
                     ..      ..     
                 r3      r2      ..     
             r1      ..      r1      r2     
         b4      r2      r1      ..      b5     
     b3      ..      ..      ..      ..      r4     
 b1      r1      r5      ..      ..      ..      r6     
     b2      ..      r2      r3      b3      ..     
         r1      ..      ..      ..      ..     
             r3      ..      ..      r4     
                 r5      r3      ..     
                     ..      ..     
                         ..     



In [32]:
np_small = np.array([[[0,0,0,0,0,0,0], 
                        [0,1,0,0,0,0,0],
                        [0,0,0,0,0,0,0],
                        [0,0,0,2,0,0,0],
                        [0,0,0,0,0,0,0],
                        [0,0,0,0,0,0,0],
                        [0,0,0,0,0,0,0]],
                        [[1,2,0,0,0,0,0],
                         [0,0,0,0,0,0,0],
                         [0,0,0,0,0,0,0],
                         [0,0,0,0,0,0,0],
                         [0,0,0,0,0,3,0],
                         [0,0,0,0,0,0,0],
                         [0,0,0,0,5,0,0]]])
board_small = MatrixBoard(np_small, 100, np_small[0].sum(), np_small[1].sum())
print(board_small.render(use_color=True, use_unicode=True))

                         ..     
                     ..      ..     
                 ..      ..      ..     
             ..      ..      ..      ..     
         ..      ..      ..      ..      b5     
     ..      ..      ..      ..      ..      ..     
 b1      r1      ..      r2      ..      ..      ..     
     b2      ..      ..      ..      b3      ..     
         ..      ..      ..      ..      ..     
             ..      ..      ..      ..     
                 ..      ..      ..     
                     ..      ..     
                         ..     



In [72]:
# state = np_small
def calculate_attack_range(state):
    domain_state = np.zeros(shape=[2,7,7], dtype=int)
    curr_player = 0
    opposite = 1
    all_dir = [HexDir.Down, HexDir.DownLeft, HexDir.DownRight, 
            HexDir.Up, HexDir.UpLeft, HexDir.UpRight]

    for player in range(2):
        for i in range(7):
            for j in range(7):
                power = state[player][i][j]
                if power == 0:
                    continue
                for d in all_dir:
                    r = i
                    q = j
                    dr = d.value.r
                    dq = d.value.q
                    for p in range(power):
                        r, q = update_r_q(r, q, dr, dq)
                        domain_state[player, r, q] = 1

    # num_attack_range = np.count_nonzero(domain_state[0])  

    domain_coodinates = set([tuple(x) for x in np.argwhere(domain_state[curr_player] > 0)])
    player_coordinates = set([tuple(x) for x in np.argwhere(state[curr_player] > 0)])
    opposite_coordinates = set([tuple(x) for x in np.argwhere(state[opposite] > 0)])
    opposite_domain = set([tuple(x) for x in np.argwhere(domain_state[opposite] > 0)])
    attack_coord = domain_coodinates & opposite_coordinates
    safeguard_coord = player_coordinates & opposite_domain
    
    # overlap_num = len(overlap_coord)
    # the total power of opposite which can be eaten by the player
    attack_power = sum([state[opposite][x] for x in attack_coord])
    
    # the total power of player which can NOT be eaten by the opposite
    safeguard_power = sum([state[curr_player][x] for x in safeguard_coord])
    print(safeguard_coord)
    return attack_power, safeguard_power

calculate_attack_range(np_small)


{(1, 1)}


(2, 1)

In [60]:
# disadvantage = (domain_state[0] - domain_state[1]).sum()
# (domain_state[0] - np_small[1])
num_attack_range = np.count_nonzero(domain_state[0])

domain_coodinates = set([tuple(x) for x in np.argwhere(domain_state[0] > 0)])
opposite_coordinates = set([tuple(x) for x in np.argwhere(np_small[1] > 0)])
overlap_coord = domain_coodinates & opposite_coordinates
 
overlap_num = len(overlap_coord)
overlap_power = sum([np_small[1][x] for x in overlap_coord])

In [38]:
domain_board = MatrixBoard(domain_state, 0, domain_state[0].sum(), domain_state[1].sum())
print(domain_board.render(use_color=True, use_unicode=True))

                         b1     
                     ..      b1     
                 b1      r1      b1     
             b1      b1      ..      b1     
         r1      r1      r1      r1      ..     
     r1      r1      r1      r1      b1      b1     
 b1      b1      ..      ..      b1      b1      b1     
     r1      r1      r1      r1      ..      ..     
         r1      r1      r1      r1      b1     
             b1      b1      b1      b1     
                 b1      r1      b1     
                     b1      b1     
                         b1     

